In [1]:
pip install pandas openpyxl

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install xlrd>=2.0.1

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import json
import os
import io

# ==========================
# CONFIGURATION
# ==========================
OUTPUT_JSON_FILE = "namaste_data.json"

# UPDATED: Keys are now all lowercase to match the Normalized Headers logic
FILES_CONFIG = [
    {
        "system": "Ayurveda",
        "keyword": "AYURVEDA", 
        "mapping": {
            "term": "namc_term",          
            "english": "name english",    
            "tm2_code": "namc_code"       
        }
    },
    {
        "system": "Siddha",
        "keyword": "SIDDHA",
        "mapping": {
            "term": "namc_term",
            "english": "short_definition", 
            "tm2_code": "namc_code"
        }
    },
    {
        "system": "Unani",
        "keyword": "UNANI",
        "mapping": {
            "term": "numc_term",
            "english": "short_definition", 
            "tm2_code": "numc_code"
        }
    },
    {
        "system": "ICD-10",
        "keyword": "ICD10",
        "mapping": {
            "term": "namc_term",
            "english": "namc_term",       
            "tm2_code": "namc_code"
        }
    }
]

def find_file_by_keyword(keyword):
    """
    Scans the current directory for a CSV or Excel file containing the keyword.
    """
    try:
        current_files = os.listdir('.')
        matches = [f for f in current_files if keyword.lower() in f.lower() and (f.endswith('.csv') or f.endswith('.xls') or f.endswith('.xlsx'))]
        if matches:
            return matches[0]
    except Exception as e:
        print(f"   ⚠️ Error scanning directory: {e}")
    return None

def read_file_smartly(file_path):
    """
    Tries multiple strategies to read the file, prioritizing Excel for .xls files.
    """
    
    # 1. Check if it's likely a binary Excel file (.xls)
    if file_path.lower().endswith('.xls'):
        try:
            # Try reading as Excel using xlrd (Required for .xls)
            return pd.read_excel(file_path, engine='xlrd')
        except ImportError:
            print(f"\n   ❌ ERROR: You are missing the 'xlrd' library required for .xls files.")
            print(f"      Please run: pip install xlrd")
            raise
        except Exception as e:
            # If it fails, maybe it's a CSV named as .xls? Fall through to CSV logic.
            print(f"   ⚠️ Excel read failed ({e}), attempting CSV read...")

    # 2. Strategy: Smart Separator Hunt (For CSVs)
    separators = [',', '\t', ';', '|']
    encodings = ['utf-8', 'latin1', 'cp1252', 'iso-8859-1']
    
    for sep in separators:
        for encoding in encodings:
            try:
                df = pd.read_csv(file_path, sep=sep, encoding=encoding, on_bad_lines='skip')
                
                # VALIDATION: Check for Binary Garbage
                # If columns contain weird characters like 'ðï', it's a binary file read as text. Reject it.
                col_str = "".join(list(df.columns))
                if "\ufffd" in col_str or "ð" in col_str or "à" in col_str:
                     continue # Skip this result, it's garbage
                
                if len(df.columns) > 1:
                    return df
            except:
                continue

    # 3. Final Fallback: Try generic Excel (for .xlsx)
    try:
        return pd.read_excel(file_path)
    except Exception as e:
        print(f"\n      [Debug] Failed to read '{file_path}'.")
        raise e

def convert_excel_to_json():
    all_records = []
    print("🚀 Starting Smart Conversion Process (Robust Mode with Verification)...")
    print(f"📂 Scanning current folder: {os.getcwd()}")
    
    # Validation Stats
    stats = {}

    for config in FILES_CONFIG:
        system_name = config["system"]
        keyword = config["keyword"]
        mapping = config["mapping"]
        stats[system_name] = {"total": 0, "unknown_terms": 0}
        
        # 1. Find the file
        file_path = find_file_by_keyword(keyword)
        
        if not file_path:
            print(f"⚠️  Skipping {system_name}: No file found with keyword '{keyword}'")
            continue
            
        print(f"📖 Processing {system_name} (Found: {file_path})...")
        
        try:
            # 2. Read File (Smart Mode)
            df = read_file_smartly(file_path)
            
            # 3. NORMALIZE HEADERS (Fixes "Unknown" issues)
            # Convert all columns to lowercase and strip spaces
            df.columns = [str(c).lower().strip() for c in df.columns]
            
            # Debug: Print found columns to help user verify
            print(f"   [Debug] Found Columns: {list(df.columns)[:5]} ...")
            
            # 4. Process Data
            file_records = []
            
            for index, row in df.iterrows():
                # Extract data safely using lowercase keys
                term = str(row.get(mapping["term"], "Unknown")).strip()
                code = str(row.get(mapping["tm2_code"], "Unknown")).strip()
                
                # Check for bad mapping
                if term == "Unknown" or term == "nan":
                    stats[system_name]["unknown_terms"] += 1
                    term = "Unknown" 

                # English fallback logic
                english_val = row.get(mapping["english"], None)
                if pd.isna(english_val) or str(english_val).strip() == "":
                    english = term
                else:
                    english = str(english_val).strip()
                
                record = {
                    "term": term,
                    "english": english,
                    "tm2_code": code,
                    "system": system_name
                }
                file_records.append(record)
            
            all_records.extend(file_records)
            stats[system_name]["total"] = len(file_records)
            print(f"   ✅ Added {len(file_records)} records from {system_name}")

        except Exception as e:
            print(f"   ❌ Critical Error processing {system_name}: {e}")

    # 5. Save Final JSON
    if all_records:
        with open(OUTPUT_JSON_FILE, 'w', encoding='utf-8') as f:
            json.dump(all_records, f, indent=4, ensure_ascii=False)
            
        print("\n===================================================")
        print("🔍 QUALITY CHECK REPORT")
        print("===================================================")
        for sys, data in stats.items():
            total = data["total"]
            unknowns = data["unknown_terms"]
            if total == 0:
                print(f"❌ {sys}: No records found.")
            elif unknowns == total:
                print(f"⚠️  {sys}: {total} records, but ALL have 'Unknown' terms. (Check Headers!)")
            elif unknowns > 0:
                print(f"⚠️  {sys}: {total} records, {unknowns} terms missing/unknown.")
            else:
                print(f"✅ {sys}: {total} records, 100% Clean.")
        
        print("===================================================")
        print(f"🎉 SUCCESS! Total {len(all_records)} records saved to '{OUTPUT_JSON_FILE}'")
        print("👉 You can now run 'server.py' to use this database.")
    else:
        print("❌ No records were processed.")

if __name__ == "__main__":
    convert_excel_to_json()

🚀 Starting Smart Conversion Process (Robust Mode with Verification)...
📂 Scanning current folder: C:\Users\Shriharsh\SIH
📖 Processing Ayurveda (Found: NATIONAL AYURVEDA MORBIDITY CODES.xls)...
   [Debug] Found Columns: ['sr no.', 'namc_id', 'namc_code', 'namc_term', 'namc_term_diacritical'] ...
   ✅ Added 2910 records from Ayurveda
📖 Processing Siddha (Found: NATIONAL SIDDHA MORBIDITY CODES.xls)...
   [Debug] Found Columns: ['sr no.', 'namc_id', 'namc_code', 'namc_term', 'tamil_term'] ...
   ✅ Added 1926 records from Siddha
📖 Processing Unani (Found: NATIONAL UNANI MORBIDITY CODES.xls)...
   [Debug] Found Columns: ['sr no.', 'numc_id', 'numc_code', 'arabic_term', 'numc_term'] ...
   ✅ Added 2522 records from Unani
📖 Processing ICD-10 (Found: NATIONAL ICD10 MORBIDITY CODES.xls)...
   [Debug] Found Columns: ['sr no.', 'namc_id', 'namc_code', 'namc_term', 'block_title'] ...
   ✅ Added 11145 records from ICD-10

🔍 QUALITY CHECK REPORT
✅ Ayurveda: 2910 records, 100% Clean.
⚠️  Siddha: 1926 

In [1]:
import pandas as pd
import json
import os
import io

# ==========================
# CONFIGURATION
# ==========================
OUTPUT_JSON_FILE = "namaste_data.json"

# UPDATED: Added mapping for 'description' columns
FILES_CONFIG = [
    {
        "system": "Ayurveda",
        "keyword": "AYURVEDA", 
        "mapping": {
            "term": "namc_term",          
            "english": "name english",    
            "tm2_code": "namc_code",
            "desc_long": "long_definition",  # <--- New
            "desc_short": "short_definition" # <--- New
        }
    },
    {
        "system": "Siddha",
        "keyword": "SIDDHA",
        "mapping": {
            "term": "namc_term",
            "english": "short_definition", 
            "tm2_code": "namc_code",
            "desc_long": "long_definition",
            "desc_short": "short_definition"
        }
    },
    {
        "system": "Unani",
        "keyword": "UNANI",
        "mapping": {
            "term": "numc_term",
            "english": "short_definition", 
            "tm2_code": "numc_code",
            "desc_long": "long_definition",
            "desc_short": "short_definition"
        }
    },
    {
        "system": "ICD-10",
        "keyword": "ICD10",
        "mapping": {
            "term": "namc_term",
            "english": "namc_term",       
            "tm2_code": "namc_code",
            "desc_long": "block_title",      # Using block title as context
            "desc_short": "chapt_name"
        }
    }
]

def find_file_by_keyword(keyword):
    try:
        current_files = os.listdir('.')
        matches = [f for f in current_files if keyword.lower() in f.lower() and (f.endswith('.csv') or f.endswith('.xls') or f.endswith('.xlsx'))]
        if matches:
            return matches[0]
    except Exception as e:
        print(f"   ⚠️ Error scanning directory: {e}")
    return None

def read_file_smartly(file_path):
    # 1. Check if it's likely a binary Excel file (.xls)
    if file_path.lower().endswith('.xls'):
        try:
            return pd.read_excel(file_path, engine='xlrd')
        except ImportError:
            print(f"\n   ❌ ERROR: Missing 'xlrd'. pip install xlrd>=2.0.1")
            raise
        except Exception:
            pass # Try CSV fallback

    # 2. Strategy: Smart Separator Hunt (For CSVs)
    separators = [',', '\t', ';', '|']
    encodings = ['utf-8', 'latin1', 'cp1252', 'iso-8859-1']
    
    for sep in separators:
        for encoding in encodings:
            try:
                df = pd.read_csv(file_path, sep=sep, encoding=encoding, on_bad_lines='skip')
                col_str = "".join(list(df.columns))
                if "\ufffd" in col_str or "ð" in col_str or "à" in col_str: continue 
                if len(df.columns) > 1: return df
            except: continue

    # 3. Final Fallback: Try generic Excel (for .xlsx)
    try:
        return pd.read_excel(file_path)
    except Exception as e:
        print(f"\n      [Debug] Failed to read '{file_path}'.")
        raise e

def clean_text(val):
    """Helper to clean up text values (remove NaNs, dashes, whitespace)"""
    if pd.isna(val): return ""
    val = str(val).strip()
    if val in ["-", "nan", "Unknown", ""]: return ""
    return val

def convert_excel_to_json():
    all_records = []
    print("🚀 Starting Smart Conversion (With Descriptions)...")
    
    stats = {}

    for config in FILES_CONFIG:
        system_name = config["system"]
        keyword = config["keyword"]
        mapping = config["mapping"]
        stats[system_name] = {"total": 0}
        
        file_path = find_file_by_keyword(keyword)
        if not file_path:
            print(f"⚠️  Skipping {system_name}: File not found.")
            continue
            
        print(f"📖 Processing {system_name}...")
        
        try:
            df = read_file_smartly(file_path)
            # Normalize headers
            df.columns = [str(c).lower().strip() for c in df.columns]
            
            file_records = []
            
            for index, row in df.iterrows():
                term = clean_text(row.get(mapping["term"]))
                code = clean_text(row.get(mapping["tm2_code"]))
                
                # English Logic
                english = clean_text(row.get(mapping["english"]))
                if not english: english = term
                
                # --- NEW: Description Logic ---
                # Try Long Definition first, then Short, then Default
                desc_long = clean_text(row.get(mapping.get("desc_long")))
                desc_short = clean_text(row.get(mapping.get("desc_short")))
                
                final_desc = ""
                if desc_long: 
                    final_desc = desc_long
                elif desc_short:
                    final_desc = desc_short
                else:
                    final_desc = "No description available."
                
                # Fallback for ICD-10 context
                if system_name == "ICD-10" and final_desc:
                     final_desc = f"Category: {final_desc}"

                if not term: term = "Unknown"
                if not code: code = "Unknown"

                record = {
                    "term": term,
                    "english": english,
                    "tm2_code": code,
                    "system": system_name,
                    "description": final_desc  # <--- Field added
                }
                file_records.append(record)
            
            all_records.extend(file_records)
            stats[system_name]["total"] = len(file_records)
            print(f"   ✅ Added {len(file_records)} records.")

        except Exception as e:
            print(f"   ❌ Error: {e}")

    if all_records:
        with open(OUTPUT_JSON_FILE, 'w', encoding='utf-8') as f:
            json.dump(all_records, f, indent=4, ensure_ascii=False)
        print(f"🎉 SUCCESS! {len(all_records)} records saved with descriptions.")
    else:
        print("❌ No records processed.")

if __name__ == "__main__":
    convert_excel_to_json()

🚀 Starting Smart Conversion (With Descriptions)...
📖 Processing Ayurveda...
   ✅ Added 2910 records.
📖 Processing Siddha...
   ✅ Added 1926 records.
📖 Processing Unani...
   ✅ Added 2522 records.
📖 Processing ICD-10...
   ✅ Added 11145 records.
🎉 SUCCESS! 18503 records saved with descriptions.
